In [401]:
import re
import xgboost as xgb
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
import pandas as pd
import numpy as np
from sklearn.model_selection import RandomizedSearchCV, GridSearchCV
import optuna as opt

In [402]:
db_test = pd.read_csv("../data/test_data.csv")

db_train = pd.read_csv("../data/train_data.csv")

In [403]:
db_train["Ticket"].describe()

count     1493
unique     454
top        181
freq       660
Name: Ticket, dtype: object

In [404]:
def wordRemover(x):
    hei = re.search("^[A-Za-z]", x)
    
    print(hei)
    if hei == None:
        return x
    else:
        return np.nan

Remove duplicates

In [405]:
db_train.drop_duplicates(inplace=True)
db_train["Ticket"].describe()

count     572
unique    454
top       208
freq        5
Name: Ticket, dtype: object

Remove words in fare and ticket

In [406]:
db_train["Ticket"] = db_train["Ticket"].apply(lambda x: wordRemover(x))
db_train["Fare"] = db_train["Fare"].apply(lambda x: wordRemover(x))

None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None


In [407]:
db_train.dropna(inplace=True)

In [408]:
db_train.info()


<class 'pandas.core.frame.DataFrame'>
Int64Index: 561 entries, 0 to 1492
Data columns (total 11 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  561 non-null    int64  
 1   Survived     561 non-null    int64  
 2   Pclass       561 non-null    int64  
 3   Sex          561 non-null    int64  
 4   SibSp        561 non-null    int64  
 5   Parch        561 non-null    int64  
 6   Ticket       561 non-null    object 
 7   Fare         561 non-null    object 
 8   Cabin        561 non-null    int64  
 9   Embarked     561 non-null    int64  
 10  Age          561 non-null    float64
dtypes: float64(1), int64(8), object(2)
memory usage: 52.6+ KB


Changing object types to int and float

In [409]:
db_train["Ticket"] = db_train["Ticket"].astype(int)
db_train["Fare"] = db_train["Fare"].astype(float)

In [410]:
db_train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 561 entries, 0 to 1492
Data columns (total 11 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  561 non-null    int64  
 1   Survived     561 non-null    int64  
 2   Pclass       561 non-null    int64  
 3   Sex          561 non-null    int64  
 4   SibSp        561 non-null    int64  
 5   Parch        561 non-null    int64  
 6   Ticket       561 non-null    int64  
 7   Fare         561 non-null    float64
 8   Cabin        561 non-null    int64  
 9   Embarked     561 non-null    int64  
 10  Age          561 non-null    float64
dtypes: float64(2), int64(9)
memory usage: 52.6 KB


In [411]:
db_train = db_train[db_train["Age"] > 0]
db_train = db_train[db_train["Age"] < 100]

In [412]:
model = xgb.XGBRegressor()
X = db_train.drop(["Age"], axis=1)
y = db_train["Age"]


In [413]:
X_testdata = db_test.drop('Age', axis=1)
y_testdata = db_test['Age']

In [414]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [415]:
model.fit(X_train, y_train)

/opt/miniconda3/envs/intro-ai/lib/python3.8/site-packages/xgboost/data.py:250: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  elif isinstance(data.columns, (pd.Int64Index, pd.RangeIndex)):


XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=1, enable_categorical=False,
             gamma=0, gpu_id=-1, importance_type=None,
             interaction_constraints='', learning_rate=0.300000012,
             max_delta_step=0, max_depth=6, min_child_weight=1, missing=nan,
             monotone_constraints='()', n_estimators=100, n_jobs=8,
             num_parallel_tree=1, predictor='auto', random_state=0, reg_alpha=0,
             reg_lambda=1, scale_pos_weight=1, subsample=1, tree_method='exact',
             validate_parameters=1, verbosity=None)

In [416]:
preds = model.predict(X_testdata)
mse = mean_squared_error(preds, y_testdata)
rmse = np.sqrt(mse)
rmse

12.611457047340792

In [417]:
#use optuna to tune the hyperparameters
def objective(trial):
    param = {
            "n_estimators" : trial.suggest_int('n_estimators', 0, 1000),
            'max_depth':trial.suggest_int('max_depth', 3, 10),
            'reg_alpha':trial.suggest_int('reg_alpha', 0, 5),
            'reg_lambda':trial.suggest_int('reg_lambda', 0, 5),
            'gamma':trial.suggest_int('gamma', 0, 5),
            'learning_rate':trial.suggest_loguniform('learning_rate',0.005,0.5),
            'colsample_bytree':trial.suggest_discrete_uniform('colsample_bytree',0.1,1,0.01),
            'subsample': trial.suggest_loguniform("subsample", 0.5, 1.0 )
        
        }
    model = xgb.XGBRegressor(**param)
    model.fit(X_train, y_train)
    preds = model.predict(X_test)
    score = mean_squared_error(y_test, preds)
    return score


study = opt.create_study(direction='minimize')
study.optimize(objective, n_trials=50)

[I 2022-10-27 21:51:45,419] A new study created in memory with name: no-name-33d91ebe-224d-4107-ae73-adc4c8cf3a3b
/var/folders/6s/94cj8dh53f76lqyqy2psnzcr0000gn/T/ipykernel_6784/4106916104.py:9: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use :func:`~optuna.trial.Trial.suggest_float` instead.
  'learning_rate':trial.suggest_loguniform('learning_rate',0.005,0.5),
/var/folders/6s/94cj8dh53f76lqyqy2psnzcr0000gn/T/ipykernel_6784/4106916104.py:10: FutureWarning: suggest_discrete_uniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use :func:`~optuna.trial.Trial.suggest_float` instead.
  'colsample_bytree':trial.suggest_discrete_uniform('colsample_bytree',0.1,1,0.01),
/var/folders/6s/94cj8dh53f76lqyqy2psnzcr0000gn/T/ipykernel_6784/4106916104.py:11: FutureWarning: suggest_loguniform has been d

In [418]:
model2 = xgb.XGBClassifier(**study.best_params)
model2.fit(X_train, y_train)

/opt/miniconda3/envs/intro-ai/lib/python3.8/site-packages/xgboost/sklearn.py:1224: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)
/opt/miniconda3/envs/intro-ai/lib/python3.8/site-packages/xgboost/data.py:250: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  elif isinstance(data.columns, (pd.Int64Index, pd.RangeIndex)):


[21:52:16] WARNING: /var/folders/nz/j6p8yfhx1mv_0grj5xl4650h0000gp/T/abs_eek2t0c4ro/croots/recipe/xgboost-split_1659548960591/work/src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=0.4,
              enable_categorical=False, gamma=0, gpu_id=-1,
              importance_type=None, interaction_constraints='',
              learning_rate=0.007845493961257775, max_delta_step=0, max_depth=3,
              min_child_weight=1, missing=nan, monotone_constraints='()',
              n_estimators=784, n_jobs=8, num_parallel_tree=1,
              objective='multi:softprob', predictor='auto', random_state=0,
              reg_alpha=1, reg_lambda=1, scale_pos_weight=None,
              subsample=0.589613875882303, tree_method='exact',
              validate_parameters=1, verbosity=None)

In [419]:
preds = model2.predict(X_testdata)
mse = mean_squared_error(preds, y_testdata)
rmse = np.sqrt(mse)
rmse

16.011195820687213